In [10]:
#!pip install pytorch
#!pip install tensorflow
#!pip install transformers

In [11]:
import transformers
import pandas as pd
import numpy as np
import json
import os
import torch

In [13]:
predict = transformers.pipeline('zero-shot-classification',model='facebook/bart-large-mnli')

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [16]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Money Market.gslides',
 'Team report        Ansgar Wiechert i6172111.gdoc',
 'Growth, debt and deficit UK, Japan, US.gslides',
 'Businessplan.gdoc',
 'Final Report Alternatice Investment.gdoc',
 'EBC2121-03-Chapter4.2-Ansgar&Nika.mov',
 'ACC - Nestle Analysis.gdoc',
 'ACC - Nestle.gsheet',
 'Colab Notebooks',
 'AirThread Valuation Case.gslides',
 'Problem_Set_3_Group_13.ipynb',
 'Programming 2021 - Project.gdoc',
 'Investments',
 'quotes-2019-nytimes.json']

In [17]:
def get_data(path):
  data = []
  with open(path) as f:
      for line in f:
          data.append(json.loads(line))
  return data

In [24]:
labels = ['climate','pollution','waste','dirty','ozone','warming','temperature']
hypothesis_template = 'This text is about {}.'

In [29]:
""" test run """
all = {}
for j in range(5):
    quote = data[j]['quotation']
    prediction = predict(quote, labels, hypothesis_template=hypothesis_template, multi_label=True)
    all[quote] = sum(prediction['scores'])


In [30]:
print(all)

{'It is not a low-income immigration,': 0.06475158804096282, 'a champion figure skater switching to roller skates.': 0.09562503406777978, 'It makes it much more difficult for him to make the compromises needed.': 0.5849343072623014, 'It puts me in a predicament,': 0.9215890374034643, 'A Pile of Leaves.': 1.9862750155152753}


In [ ]:
def get_labels(data,dict,score_thresh):
    N = len(data)
    trues = {}
    wrongs = {}
    for i in range(N):
      quote = data[i]['quotation']
      prediction = predict(quote, labels, hypothesis_template=hypothesis_template, multi_label=True)
      score = sum(prediction['scores'])
      if score > score_thresh:
        trues[quote] = (score,1) # 1 for that the quote deals with the defined topic
      else:
        wrongs[quote] = (score,0) # 0 for that the quote does not deal with the topic
        
    return trues, wrongs
